In [9]:
import cv2 as cv
import numpy as np
import os
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from joblib import dump, load
from sklearn.neural_network import MLPClassifier
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from joblib import dump, load
from PIL import Image, ImageFilter, ImageChops
from skimage import feature

### Read images

In [2]:
def read_images(input_path):
    """
    Read images in the input_path, 
    save image, patient of each image and the class (group/labels)
    
    Params: 
    input_path = path to the original images 
    
    Return:
    images = list of all images
    labels = list with class for each image
    """
   
    # Lists to save images, patients and labels
    images = []
    labels = []

    # Browse input path
    for class_dir in os.listdir(input_path):
        class_path = os.path.join(input_path, class_dir)

        # If it is a directory 
        if os.path.isdir(class_path):      

            for image_file in os.listdir(class_path):
                image_path = os.path.join(class_path, image_file)

                image = cv.imread(image_path, cv.IMREAD_GRAYSCALE)

                # Append image, patient id and class to list
                images.append(image)            
                labels.append(class_dir)    
                              
    return (images, labels)

In [3]:
x_train, y_train = read_images("/home/vsa/ufpr/cb/ufpr-bioinspired-computing/dataset/train")
#x_test, y_test = read_images("/home/vsa/ufpr/cb/ufpr-bioinspired-computing/dataset/test")

### Preprocessing

### Extract features

In [9]:
# Extract grayscale features
def extract_grayscale(data):
    
    ft_gray = []
    
    for image in data:
        
        shape = image.shape
        ft_gray.append(np.reshape(image, (shape[0]*shape[1])))
    
    return ft_gray

In [4]:
# Extract feature: length of the edges
def extract_edges_len(data):
    
    ft_edges_len = []
    
    for image in data:
        
        edges = feature.canny(image)
        
        # Extract the edge pixels
        edge_pixels = edges.nonzero()

        # Calculate the edge length for each edge pixel
        edge_lengths = []
        
        for i, j in zip(*edge_pixels):
            # Get the coordinates of the current pixel
            x, y = j, i

            # Check if there's a next pixel in the edge segment
            if x > 0:
                # Get the coordinates of the next pixel
                x2 = edge_pixels[0][x - 1]
                y2 = edge_pixels[1][x - 1]
            else:
                # Handle the edge case where there's no next pixel
                x2 = edge_pixels[0][-1]
                y2 = edge_pixels[1][-1]

            # Calculate the distance between the two pixels
            distance = np.sqrt((x2 - x)**2 + (y2 - y)**2)

            # Add the distance to the list of edge lengths
            edge_lengths.append(distance)
            
        ft_edges_len.append(edge_lengths) 
      
    return ft_edges_len

In [ ]:
# Extract features: curvature of the edges
from scipy.interpolate import CubicBezier
def extract_edges_curv(data):
    edges = feature.canny(image)
    

In [17]:
# Extract features: LBP

def extract_lbp(data, eps=1e-7, points=24, radius=8):
    
    ft_lbp = []
    
    for image in data:
        
        lbp = feature.local_binary_pattern(image,
                                           points,
                                           radius,
                                           method="uniform")

        (hist, _) = np.histogram(lbp.ravel(), 
                                 bins = np.arange(0, points + 3),
                                                 range=(0, points + 2))

        # normalize the histogram
        hist = hist.astype("float")
        hist /= (hist.sum() + eps)
        
        ft_lbp.append(hist)

    # return the histogram of Local Binary Patterns
    return ft_lbp

In [6]:
ft_gray = extract_grayscale(x_train)

In [5]:
ft_edges_len = extract_edges_len(x_train)

In [23]:
ft_lbp = extract_lbp(x_train)

In [11]:
dump(ft_gray, 'ft_gray.joblib')
dump(ft_edges_len, 'ft_edges_len.joblib')

['ft_mean.joblib']

In [ ]:
ft_gray = load('ft_gray.joblib')
ft_edges_len = load('ft_mean.joblib')

In [3]:
ft_gray = load('ft_gray.joblib')


In [ ]:
dump(ft_edges_len, 'ft_edges_len.joblib')

In [ ]:
# Grayscale Pixel Values done
# length of the edges done
# curvature of edges

Get feature names and table

In [6]:
max_edges_count = 0
for sample in ft_edges_len:
    edges_count = len(sample)
    
    if edges_count > max_edges_count:
        max_edges_count = edges_count
        


In [4]:
names = []

names_gray = [f'gray{i}' for i in range(len(ft_gray[0]))]
names_edges = [f'edges{i}' for i in range(max_edges_count)]
names_lbp = [f'lbp{i}' for i in range(len(ft_lbp[0]))]
names.append(names_gray)
names.append(names_edges)
names.append(names_lbp)


In [ ]:
df = pd.DataFrame({'image': images, 'label': labels})

# Create a LabelEncoder object
le = LabelEncoder()
le.fit(df['label'])
df['label'] = le.transform(df['label'])

df = df.reset_index()

### PSO